<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width = 400, align = "center"></a>
# <center>K-Nearest Neighbors</center>

En esta práctica de laboratorio, cargará un conjunto de datos del cliente, ajustará los datos y **usará vecinos más cercanos// K-Nearest Neighbors** para predecir un punto de datos. Pero, ¿qué es **K-Nearest Neighbors**?


**K-Nearest Neighbors** Es un algoritmo para el aprendizaje supervisado. Donde los datos están 'entrenados' con puntos de datos correspondientes a su clasificación. Una vez que se va a predecir un punto, tiene en cuenta los puntos "K" más cercanos para determinar su clasificación.

### Here's an visualization of the K-Nearest Neighbors algorithm.
### Aquí hay una visualización del algoritmo K-Nearest Neighbours.

<img src = "https://ibm.box.com/shared/static/mgkn92xck0z05v7yjq8pqziukxvc2461.png">

En este caso, tenemos puntos de datos de Clase A y B. Queremos predecir qué es la estrella (punto de datos de prueba). Si consideramos un valor k de 3 (3 puntos de datos más cercanos) obtendremos una predicción de la Clase B. Sin embargo, si consideramos un valor k de 6, obtendremos una predicción de la Clase A.

En este sentido, es importante considerar el valor de k. Pero, con suerte, a partir de este diagrama, debe tener una idea de lo que es el algoritmo de Vecinos **K-Nearest Neighbors**. Considera los vecinos más cercanos (puntos) 'K' cuando predice la clasificación del punto de prueba.

Carguemos las bibliotecas requeridas

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

### About dataset

Imagine que un proveedor de telecomunicaciones ha segmentado su base de clientes por patrones de uso del servicio, categorizando a los clientes en cuatro grupos. Si los datos demográficos se pueden utilizar para predecir la pertenencia al grupo, la empresa puede personalizar las ofertas para clientes potenciales individuales. Es un problema de clasificación. Es decir, dado el conjunto de datos, con etiquetas predefinidas, necesitamos construir un modelo que se use para predecir la clase de un caso nuevo o desconocido.

El ejemplo se centra en el uso de datos demográficos, como la región, la edad y el matrimonio, para predecir patrones de uso.

El campo de destino, llamado __custcat__, tiene cuatro valores posibles que corresponden a los cuatro grupos de clientes, de la siguiente manera:
   1- Servicio básico               Basic Service
   2- Servicio electrónico          E-Service
   3- Servicio Plus                 Plus Service
   4- Servicio total                Total Service

Nuestro objetivo es construir un clasificador, para predecir la clase de casos desconocidos. Usaremos un tipo específico de clasificación llamado  **K nearest neighbour** vecino más cercano.


Vamos a descargar el conjunto de datos. Para descargar los datos, utilizaremos! Wget para descargarlos de IBM Object Storage.

In [ ]:
!wget -O teleCust1000t.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/teleCust1000t.csv

__ ¿Sabía que? __ Cuando se trata de Machine Learning, es probable que trabaje con grandes conjuntos de datos. Como empresa, ¿dónde puede alojar sus datos? IBM ofrece una oportunidad única para las empresas, con 10 Tb de IBM Cloud Object Storage: [Sign up now for free](http://cocl.us/ML0101EN-IBM-Offer-CC)

### Load Data From CSV File  

In [ ]:
df = pd.read_csv('teleCust1000t.csv')
df.head()

# Data Visualization and Anylisis 



#### Veamos cuántos de cada clase hay en nuestro conjunto de datos

In [ ]:
df['custcat'].value_counts()

#### Servicio 281 Plus, 266 servicio básico, 236 servicio total y 217 clientes de servicio electrónico

Puede explorar fácilmente sus datos utilizando técnicas de visualización:

In [ ]:
df.hist(column='income', bins=50)

### Feature set

Lets defind feature sets, X:
Definamos conjuntos de características, X:

In [ ]:
df.columns

Para usar la biblioteca scikit-learn, tenemos que convertir el marco de datos de Pandas a una matriz de Numpy:

To use scikit-learn library, we have to convert the Pandas data frame to a Numpy array:

In [ ]:
X = df[['region', 'tenure','age', 'marital', 'address', 'income', 'ed', 'employ','retire', 'gender', 'reside']] .values  #.astype(float)
X[0:5]


¿Cuáles son nuestras etiquetas (**lables**)?

In [1]:
y = df['custcat'].values
y[0:5]

NameError: name 'df' is not defined

## Normalize Data 

La estandarización de datos proporciona a los datos media cero y varianza unitaria, es una buena práctica, especialmente para algoritmos como KNN que se basa en la distancia de los casos:


In [ ]:
X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))
X[0:5]

### Train Test Split  

La precisión fuera de la muestra es el porcentaje de predicciones correctas que el modelo hace sobre los datos en los que el modelo NO ha sido entrenado. Hacer un tren y probar en el mismo conjunto de datos probablemente tendrá una baja precisión fuera de la muestra, debido a la probabilidad de un sobreajuste.

Es importante que nuestros modelos tengan una alta precisión fuera de la muestra, porque el propósito de cualquier modelo, por supuesto, es hacer predicciones correctas sobre datos desconocidos. Entonces, ¿cómo podemos mejorar la precisión fuera de la muestra? Una forma es utilizar un enfoque de evaluación llamado Train / Test Split.
La división de entrenamiento / prueba implica dividir el conjunto de datos en conjuntos de entrenamiento y prueba, respectivamente, que son mutuamente excluyentes. Después de lo cual, entrena con el conjunto de entrenamiento y prueba con el conjunto de prueba.

Esto proporcionará una evaluación más precisa de la precisión fuera de la muestra porque el conjunto de datos de prueba no es parte del conjunto de datos que se ha utilizado para entrenar los datos. Es más realista para problemas del mundo real.


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

# Classification 

## K nearest neighbor (K-NN)

#### Import library 

Classifier implementing the k-nearest neighbors vote.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

### Training

Lets start the algorithm with k=4 for now:

In [ ]:
k = 4
#Train Model and Predict  
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
neigh

### Predicting
we can use the model to predict the test set:

In [ ]:
yhat = neigh.predict(X_test)
yhat[0:5]

### Accuracy evaluation
En la clasificación de múltiples capas **(multilabel classification)**, la función __accuracy classification score__ calcula la precisión del subconjunto. Esta función es igual a la función jaccard_similarity_score. Básicamente, calcula cómo coinciden las etiquetas reales y las predichas en el conjunto de prueba.


In [ ]:
from sklearn import metrics
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat))

## Practice
¿Puedes volver a construir el modelo, pero esta vez con k=6?

In [ ]:
# write your code here




Double-click __here__ for the solution.

<!-- Your answer is below:
    
    
k = 6
neigh6 = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
yhat6 = neigh6.predict(X_test)
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh6.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat6))

-->

#### What about other K? #### ¿Qué pasa con otras K?
K en KNN, es el número de vecinos más cercanos a examinar. Se supone que debe ser especificado por el usuario. Entonces, ¿cómo elegimos K correcto?
La solución general es reservar una parte de sus datos para probar la precisión del modelo. Luego elija k = 1, use la parte de entrenamiento para modelar y calcule la precisión de la predicción usando todas las muestras en su conjunto de prueba. Repita este proceso, aumentando la k, y vea qué k es la mejor para su modelo.

Podemos calcular la precisión de KNN para diferentes Ks.


In [ ]:
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

mean_acc

#### Plot  model accuracy  for Different number of Neighbors 

In [ ]:
plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.legend(('Accuracy ', '+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Nabors (K)')
plt.tight_layout()
plt.show()

In [ ]:
print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1) 

## Want to learn more? ¿Querer aprender más?

IBM SPSS Modeler es una plataforma analítica integral que tiene muchos algoritmos de aprendizaje automático. Ha sido diseñado para brindar inteligencia predictiva a las decisiones tomadas por individuos, por grupos, por sistemas, por toda su empresa. Una prueba gratuita está disponible a través de este curso, disponible aquí [SPSS Modeler](http://cocl.us/ML0101EN-SPSSModeler)

Además, puede usar Watson Studio para ejecutar estos portátiles más rápido con conjuntos de datos más grandes. Watson Studio es la solución de nube líder de IBM para científicos de datos, creada por científicos de datos. Con los portátiles Jupyter, RStudio, Apache Spark y bibliotecas populares preempaquetadas en la nube, Watson Studio permite a los científicos de datos colaborar en sus proyectos sin tener que instalar nada. Únase a la comunidad de rápido crecimiento de los usuarios de Watson Studio hoy con una cuenta gratuita en [Watson Studio](https://cocl.us/ML0101EN_DSX)


### Thanks for completing this lesson!

Notebook created by: <a href = "https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>

<hr>
Copyright &copy; 2018 [Cognitive Class](https://cocl.us/DX0108EN_CC). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).​